In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.parallel
import random
import struct
import matplotlib.pyplot as plt
import parameters as var #Configuration and coarsening parameters
var.init() #initializes parameters
import utils as ut #Some utility functions 
import loss_function as lf #Custom loss function

import operators_torch as op #Interpolator and prolongator given a set of test vectors
from opendataset import ConfsDataset #class for opening gauge confs
import model as mod #import machine learning model

#--------Most likely I will need some of these in the future-------#
#import matplotlib.pyplot as plt
#import torch.optim as optim
#import torch.utils.data
#import torchvision.datasets as dset
#import torchvision.transforms as transforms
#import torchvision.utils as vutils
var.print_parameters()
device = (
    "cuda"
    if var.NGPU>0
    else "cpu"
)
print(f"Using {device} device")

*********** Configuration parameters ***********
* β=2, Nx=32, Nt=32
* Lattice sites=2048
* m0=-0.1884
* blocks_x=2, blocks_t=2 (for the aggregation)
* Nv=14
* Number of confs=1000
* Confs used for training=900
************************************************
Using cuda device


In [ ]:
"""
Loading the configurations and the near-kernel test vectors
We split train and test set
"""
dataset = ConfsDataset()                     
total_len = len(dataset)                    
train_len = int(var.TRAIN_PROP * total_len) 
test_len  = total_len - train_len   
torch.manual_seed(42)                       # <-- any integer you like

train_set, test_set = torch.utils.data.random_split(
    dataset,
    [train_len,  test_len]          # lengths in the same order
)

workers    = 2
# Batch size
batch_size = 100

#train dataloader
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,          # we usually want shuffling *only* for training
    num_workers=workers,
    pin_memory=True
)

#test dataloader
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=workers,
    pin_memory=True
)

#----returns a tensor of size [ [batch_size,4,Nx,Nt], [batch_size,Nv,2,Nx,Nt], [batch_size]]----#
#    The first entry is real and second one complex. The last entry has the indices of the confs.
#first_batch = next(iter(train_loader)) 
#--------------------------------------

device = var.DEVICE
#first_batch[0][0].shape
#print("Re(U0)",first_batch[0][0][0,0,0])
#print("Re(U1)",first_batch[0][0][1,0,0])
#print("Im(U0)",first_batch[0][0][2,0,0])
#print("Im(U1)",first_batch[0][0][3,0,0])

In [ ]:
"""
Custom weights initialization
"""  
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02) #nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0) #nn.init.constant_(m.bias.data, 0.0)

In [ ]:
"""
Declare the model
"""
model = mod.TvGenerator(var.NGPU).to(device)
if (device.type == 'cuda') and (var.NGPU > 1):
    model = nn.DataParallel(model, list(range(var.NGPU)))

# Apply the ``weights_init`` function to randomly initialize all weights
#  to ``mean=0``, ``stdev=0.02``.
model.apply(weights_init)

# Print the model
print(model)

In [ ]:
# Learning rate for optimizers
lr = 0.001
# Beta1 hyperparameter for Adam optimizers
beta1 = 0.9
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(beta1, 0.999))
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3,momentum=0.9)

In [ ]:
losses = []
def train(dataloader, model, optimizer):
    """
    Trains the model based on the test loader
    Parameters
    ----------
    dataloader : torch.utils.data.DataLoader
        DataLoader that yields the training loader:
        (confs_batch, near_kernel)
    model : torch.nn.Module
        The model to evaluate.
    Appends loss to losses (list declared outside the function)
    """
    model.train()
    criterion = lf.CustomLossTorch().to(device)               # instantiate once, reuse
    for batch_id, batch in enumerate(dataloader):
        # -------------------------------------------------
        # Load the data
        # -------------------------------------------------
        confs_batch   = batch[0].to(device)            # shape (B, …)
        near_kernel   = batch[1].to(device)            # shape (B, NV, 2, NT, NX)

        # -------------------------------------------------
        # Forward pass of the model → predicted test vectors
        # -------------------------------------------------
        # model returns a real‑valued tensor of shape [B, 4*NV, NT, NX]
        pred = model(confs_batch)                     # still a torch Tensor
        # -------------------------------------------------
        # Reshape / convert to complex dtype
        # -------------------------------------------------
        # Example: real/imag in 4 channels (Re0, Re1, Im0, Im1)
        B = pred.shape[0]
        pred = pred.view(B, var.NV, 4, var.NT, var.NX)      # (B,NV,4,NT,NX)

        # Build a complex tensor of shape (B, NV, 2, NT, NX)
        #   channel 0 → real part of component 0
        #   channel 1 → real part of component 1
        #   channel 2 → imag part of component 0
        #   channel 3 → imag part of component 1
        real = torch.stack([pred[:,:, 0], pred[:,:, 1]], dim=2)   # (B,NV,2,NT,NX)
        imag = torch.stack([pred[:,:, 2], pred[:,:, 3]], dim=2)   # (B,NV,2,NT,NX)
        pred_complex = torch.complex(real, imag)                  # (B,NV,2,NT,NX)
        # -------------------------------------------------
        # Compute loss (still on the same device)
        # -------------------------------------------------
        #We assemble P, P^+ with the SAP test vectors and find other vectors that are similar to them.
        loss = criterion(near_kernel, pred_complex)   # loss is a scalar Tensor

        # -------------------------------------------------
        # Back‑propagation
        # -------------------------------------------------
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # -------------------------------------------------
        # Logging
        # -------------------------------------------------
        loss_val = loss.item()
        current = (batch_id + 1) * B
        losses.append(loss_val)
        print(f"loss: {loss_val:>7f}  [{current:>5d}/{train_len:>5d}]") 

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, optimizer)
print("Done!")

In [ ]:
fig = plt.figure(dpi=150)
plt.plot(np.arange(epochs*(train_len//batch_size)),losses)
plt.title("Batch size = {0}, No. of examples = {1}".format(batch_size,train_len))
plt.ylabel("Loss (training)")
plt.xlabel("Epoch per batch")
plt.show()

# Check loss on test set

In [ ]:
def evaluate(dataloader, model, device, criterion=None):
    """
    Run a forward pass on the whole test set and return
    the mean loss.

    Parameters
    ----------
    dataloader : torch.utils.data.DataLoader
        DataLoader that yields the same tuple format as the training loader:
        (confs_batch, near_kernel)
    model : torch.nn.Module
        The model to evaluate. It will be switched to ``eval`` mode for the
        duration of the call and restored to its previous mode afterwards.
    device : torch.device
        Where the tensors should live (e.g. ``torch.device('cuda')``).
    criterion : torch.nn.Module, optional
        A loss object that implements ``__call__(target, prediction)``.
        If ``None`` a fresh ``lf.CustomLossTorch`` instance will be created
        on the supplied ``device`` (the same as in the training loop).

    Returns
    -------
    avg_loss : float
        Mean loss over all batches (scalar).
    batch_losses : list[float]
        Individual batch losses.
    """
    # ------------------------------------------------------------------
    # Put the model in eval mode – turns off dropout, batch‑norm updates
    # ----------------------------------------------------------------
    was_training = model.training          # remember the original state
    model.eval()

    # ------------------------------------------------------------------
    # Build (or reuse) the loss object on the proper device
    # ------------------------------------------------------------------
    if criterion is None:
        criterion = lf.CustomLossTorch().to(device)

    # ------------------------------------------------------------------
    # Containers for the per‑batch losses
    # ------------------------------------------------------------------
    batch_losses = []

    # ------------------------------------------------------------------
    # No gradient tracking – saves memory and speeds up the forward pass
    # ------------------------------------------------------------------
    with torch.no_grad():
        for batch_id, batch in enumerate(dataloader):
            # -------------------------------------------------
            # Load the data (move to the same device as the model)
            # -------------------------------------------------
            confs_batch = batch[0].to(device)          # (B, …)
            near_kernel = batch[1].to(device)          # (B, NV, 2, NT, NX)

            # -------------------------------------------------
            # Forward pass – exactly the same reshaping as in training
            # -------------------------------------------------
            pred = model(confs_batch)                  # (B, 4*NV, NT, NX)

            B = pred.shape[0]
            pred = pred.view(B, var.NV, 4, var.NT, var.NX)   # (B,NV,4,NT,NX)

            # Build complex tensor (B,NV,2,NT,NX)
            real = torch.stack([pred[:, :, 0], pred[:, :, 1]], dim=2)   # (B,NV,2,NT,NX)
            imag = torch.stack([pred[:, :, 2], pred[:, :, 3]], dim=2)   # (B,NV,2,NT,NX)
            pred_complex = torch.complex(real, imag)

            # -------------------------------------------------
            # Compute loss (still on the same device)
            # -------------------------------------------------
            loss = criterion(near_kernel, pred_complex)   # scalar Tensor

            # -------------------------------------------------
            # Store loss value (as a Python float)
            # -------------------------------------------------
            loss_val = loss.item()
            batch_losses.append(loss_val)

    # ------------------------------------------------------------------
    # Compute the mean loss over the whole set
    # ------------------------------------------------------------------
    batch_losses = np.array(batch_losses)
    avg_loss = np.mean(batch_losses) 
    std_err = np.std(batch_losses)/np.sqrt(len(batch_losses))

    # ------------------------------------------------------------------
    # Restore the original training/eval state of the model
    # ------------------------------------------------------------------
    model.train(was_training)

    return avg_loss, std_err, batch_losses

In [ ]:
test_loss, dtest_loss, test_batch_losses = evaluate(test_loader, model, device)
print(f'Test average loss: {test_loss:.6f} +- {dtest_loss:.6f}')

In [ ]:
fig = plt.figure(dpi=150)
plt.plot(np.arange(len(test_batch_losses)),test_batch_losses,linestyle='',marker='o',markersize=10)
plt.title("Batch size = {0}, No. of examples = {1}".format(batch_size,test_len))
plt.ylabel("Loss (test)")
plt.xlabel("ID")
plt.show()

In [ ]:
def SavePredictions(dataloader, model, device):
    """
    Saves test vectors predicted with the model into binary files.
    One file per test vector. The data layout is the same as for
    the near-kernel vectors used for the training.
    x, t, μ, Re(Uμ), Im(Uμ)
    """
    with torch.no_grad():
        for batch_id, batch in enumerate(dataloader):
            confs_batch = batch[0].to(device)          # (B, …)
            pred = model(confs_batch)                  # (B, 4*NV, NT, NX)
            confsID = batch[2]

            B = pred.shape[0]
            pred = pred.view(B, var.NV, 4, var.NT, var.NX)   # (B,NV,4,NT,NX)

            # Build complex tensor (B,NV,2,NT,NX)
            real = torch.stack([pred[:, :, 0], pred[:, :, 1]], dim=2)   # (B,NV,2,NT,NX)
            imag = torch.stack([pred[:, :, 2], pred[:, :, 3]], dim=2)   # (B,NV,2,NT,NX)
            pred_complex = torch.complex(real, imag).detach().cpu().numpy()
            for i in range(len(confs_batch)):
                for tv in range(var.NV):
                    file_path = "fake_tv/b{0}_{1}x{2}/{3}/conf{4}_fake_tv{5}.tv".format(var.BETA,var.NX,var.NT,var.M0_FOLDER,confsID[i],tv)
                    fmt = "<3i2d"
                    with open(file_path, "wb") as f:
                        for x in range(var.NX):
                            for t in range(var.NT):
                                for mu in range(2):
                                    value = pred_complex[i,tv,mu,t,x]
                                    Re = np.real(value)
                                    Im = np.imag(value)
                                    data = struct.pack(fmt, int(x), int(t), int(mu), float(Re), float(Im))
                                    f.write(data)

In [ ]:
#SavePredictions(test_loader, model, device)